In [63]:
import pandas as pd
import json

def process_trades(json_file, ticker, exchange, algo=None, interval=None, mode='all'):
    # Load the data from the JSON file
    target_user_id = "6673d10b68af094502c93931"
    with open(json_file) as f:
        data = json.load(f)

    # Convert the JSON data to a DataFrame
    df = pd.json_normalize(data)

    # Define the start and end times for filtering
    start_time = pd.to_datetime('13:30:00').time()
    end_time = pd.to_datetime('20:00:00').time()

    # Filter alerts for the given ticker and interval
    alerts = df[
        (df['symbol'] == ticker) &
        (df['user_id.$oid'] == target_user_id)
    ]
    
    if interval:
        alerts = alerts[alerts['interval'] == interval]
    
    # If the mode is not 'algo_summary', apply the algo filter
    if algo and mode != 'algo_summary':
        alerts = alerts[alerts['algo'] == algo]
    
    # Extract the nested datetime value and convert it to datetime
    alerts['alerted_at'] = pd.to_datetime(alerts['alerted_at.$date'])

    # Ensure that the conversion was successful
    if alerts['alerted_at'].isna().any():
        print("Warning: Some 'alerted_at' values could not be converted to datetime.")
    
    # Filter based on alerted_at between start and end times
    if exchange == 'SMART':
        filtered_alerts = alerts[(alerts['alerted_at'].dt.time >= start_time) & (alerts['alerted_at'].dt.time <= end_time)]
    else:
        filtered_alerts = alerts
    # Sort the alerts by date in descending order
    filtered_alerts = filtered_alerts.sort_values(by='alerted_at', ascending=False)
    
    # Remove the last trade of each day
    filtered_alerts['date'] = filtered_alerts['alerted_at'].dt.date
    #display(filtered_alerts)
    #filtered_alerts = filtered_alerts.groupby(['algo', 'date']).apply(lambda x: x.iloc[1:]).reset_index(drop=True)
    
    # Define the function to calculate profit/loss
    def calculate_profit_loss(row):
        if row['order_status'] in ["order_failed", "order_running"]:
            return row

        buying_total = 0
        selling_total = 0

        for event in row['events']:
            side = event['side']
            contracts = event['contracts']
            price = float(event['price'])

            if side == 'SELL':
                selling_total += contracts * price
            elif side == 'BUY':
                buying_total += contracts * price

        profit_or_loss = selling_total - buying_total
        row['profitOrLoss'] = profit_or_loss
        row['profitOrLossPercentage'] = (profit_or_loss * 100) / 500
        return row

    # Apply the function to calculate profit/loss for each trade
    filtered_alerts = filtered_alerts.apply(calculate_profit_loss, axis=1)

    if mode == 'daily':
        # Group by date and calculate daily profit
        daily_profit = filtered_alerts.groupby('date').agg(
            profitOrLoss=('profitOrLoss', 'sum'),
            numberOfTrades=('symbol', 'size')
        ).reset_index()
        
        daily_profit['profitOrLossPercentage'] = daily_profit['profitOrLoss'].apply(lambda x: f"{(x * 100 / 500):.2f}%")

        # Display the daily profits
        pd.set_option('display.max_rows', None)
        display(daily_profit)
        
        # Print total profit for the period
        total_daily_profit = daily_profit['profitOrLoss'].sum()
        total_trades = daily_profit['numberOfTrades'].sum()
        print(f"Total Daily Profit: {total_daily_profit}")
        print(f"Total Number of Trades: {total_trades}")
        
    elif mode == 'overall':
        # Calculate total profit and percentage
        total_profit = filtered_alerts['profitOrLoss'].sum()
        percentage_total_profit = (total_profit * 100) / 500  # Assuming 500 as total investment
        total_trades = filtered_alerts.shape[0]
        
        # Print calculated values
        print(f"Total Profit: {total_profit}")
        print(f"Percentage Total Profit: {percentage_total_profit}%")
        print(f"Total Number of Trades: {total_trades}")
    
    elif mode == 'all':
        # Print calculated values
        columns_to_select = ['symbol', 'alerted_at', 'alert_type', 'profitOrLoss', 'profitOrLossPercentage', 'entry.price', 'exit.price', 'order_status', 'algo', 'interval']
        
        filtered_alerts = filtered_alerts[columns_to_select]
        display(filtered_alerts)
    
    elif mode == 'algo_summary':
        # Group by algorithm and calculate total profit/loss and profit/loss percentage
        algo_summary = filtered_alerts.groupby('algo').agg(
            totalProfitOrLoss=('profitOrLoss', 'sum'),
            numberOfTrades=('symbol', 'size')
        ).reset_index()
        
        # Calculate profit/loss percentage based on total investment (assuming 500 as total investment per algo)
        algo_summary['profitOrLossPercentage'] = algo_summary['totalProfitOrLoss'].apply(lambda x: f"{(x * 100 / 500):.2f}%")
        
        # Display the summary table
        display(algo_summary)
        # Print total profit and total trades
        total_profit = algo_summary['totalProfitOrLoss'].sum()
        total_trades = algo_summary['numberOfTrades'].sum()
        print(f"Total Profit Across All Algorithms: {total_profit}")
        print(f"Total Number of Trades Across All Algorithms: {total_trades}")


In [64]:

#MARA, CLSK, COIN, RIOT, HUT, AMC, GME, AI, UPST, NKLA, KOSS, QS, DJT, CVNA, AFRM, PLTR, SOFI, PLUG, BLINK, NIO, TLRY, LCID, BYND, JWN, JMIA, TSLA
#TSLA - 5, EURUSD - 10, XAUUSD - 10, QQQ - 15, SPX - 15, SPY - 3_, SPY - 3, SPY - 15
# AFRM-Algo-1-3M-LS
ticker = 'MARA'
algo = 'Zero-AI-1'
slippage = '5'
exchange = 'SMART'
print("======Processing algo_summary=====")
process_trades('project_zero.running_trades.json', ticker, exchange, algo, slippage, 'algo_summary')
#print("======Processing daily=====")
process_trades('project_zero.running_trades.json', ticker, exchange,  algo, slippage, 'daily')
print("======Processing all=====")
process_trades('project_zero.running_trades.json', ticker, exchange,  algo, slippage, "all")

======Processing algo_summary=====


KeyError: 'user_info'

In [5]:
import pandas as pd
import json

def process_trades_2(json_file, ticker, exchange, algo=None, interval=None, mode='all'):
    # Load the data from the JSON file
    with open(json_file) as f:
        data = json.load(f)

    # Convert the JSON data to a DataFrame
    df = pd.json_normalize(data)

    # Define the start and end times for filtering
    start_time = pd.to_datetime('13:30:00').time()
    end_time = pd.to_datetime('20:00:00').time()

    # Filter alerts for the given ticker and interval
    alerts = df[(df['symbol'] == ticker)]
    
    if interval:
        alerts = alerts[alerts['interval'] == interval]
    
    # If the mode is not 'algo_summary' or 'ticker_summary', apply the algo filter
    if algo and mode not in ['algo_summary', 'ticker_summary']:
        alerts = alerts[alerts['algo'] == algo]
    
    # Extract the nested datetime value and convert it to datetime
    alerts['alerted_at'] = pd.to_datetime(alerts['alerted_at.$date'])

    # Ensure that the conversion was successful
    if alerts['alerted_at'].isna().any():
        print("Warning: Some 'alerted_at' values could not be converted to datetime.")
    
    # Filter based on alerted_at between start and end times
    if exchange == 'SMART':
        filtered_alerts = alerts[(alerts['alerted_at'].dt.time >= start_time) & (alerts['alerted_at'].dt.time <= end_time)]
    else:
        filtered_alerts = alerts
    # Sort the alerts by date in descending order
    filtered_alerts = filtered_alerts.sort_values(by='alerted_at', ascending=False)
    
    # Remove the last trade of each day
    filtered_alerts['date'] = filtered_alerts['alerted_at'].dt.date

    # Define the function to calculate profit/loss
    def calculate_profit_loss(row):
        if row['order_status'] in ["order_failed", "order_running"]:
            return row

        buying_total = 0
        selling_total = 0

        for event in row['events']:
            side = event['side']
            contracts = event['contracts']
            price = float(event['price'])

            if side == 'SELL':
                selling_total += contracts * price
            elif side == 'BUY':
                buying_total += contracts * price

        profit_or_loss = selling_total - buying_total
        row['profitOrLoss'] = profit_or_loss
        row['profitOrLossPercentage'] = (profit_or_loss * 100) / 500
        return row

    # Apply the function to calculate profit/loss for each trade
    filtered_alerts = filtered_alerts.apply(calculate_profit_loss, axis=1)

    if mode == 'daily':
        # Group by date and calculate daily profit
        daily_profit = filtered_alerts.groupby('date').agg(
            profitOrLoss=('profitOrLoss', 'sum'),
            numberOfTrades=('symbol', 'size')
        ).reset_index()
        
        daily_profit['profitOrLossPercentage'] = daily_profit['profitOrLoss'].apply(lambda x: f"{(x * 100 / 500):.2f}%")

        # Display the daily profits
        pd.set_option('display.max_rows', None)
        display(daily_profit)
        
        # Print total profit for the period
        total_daily_profit = daily_profit['profitOrLoss'].sum()
        total_trades = daily_profit['numberOfTrades'].sum()
        print(f"Total Daily Profit: {total_daily_profit}")
        print(f"Total Number of Trades: {total_trades}")
        
    elif mode == 'overall':
        # Calculate total profit and percentage
        total_profit = filtered_alerts['profitOrLoss'].sum()
        percentage_total_profit = (total_profit * 100) / 500  # Assuming 500 as total investment
        total_trades = filtered_alerts.shape[0]
        
        # Print calculated values
        print(f"Total Profit: {total_profit}")
        print(f"Percentage Total Profit: {percentage_total_profit}%")
        print(f"Total Number of Trades: {total_trades}")
    
    elif mode == 'all':
        # Print calculated values
        columns_to_select = ['symbol', 'alerted_at', 'alert_type', 'profitOrLoss', 'profitOrLossPercentage', 'entry.price', 'exit.price', 'order_status', 'algo', 'interval']
        
        filtered_alerts = filtered_alerts[columns_to_select]
        display(filtered_alerts)
    
    elif mode == 'algo_summary':
        # Group by algorithm and calculate total profit/loss and profit/loss percentage
        algo_summary = filtered_alerts.groupby('algo').agg(
            totalProfitOrLoss=('profitOrLoss', 'sum'),
            numberOfTrades=('symbol', 'size')
        ).reset_index()
        
        # Calculate profit/loss percentage based on total investment (assuming 500 as total investment per algo)
        algo_summary['profitOrLossPercentage'] = algo_summary['totalProfitOrLoss'].apply(lambda x: f"{(x * 100 / 500):.2f}%")
        
        # Display the summary table
        display(algo_summary)
        # Print total profit and total trades
        total_profit = algo_summary['totalProfitOrLoss'].sum()
        total_trades = algo_summary['numberOfTrades'].sum()
        print(f"Total Profit Across All Algorithms: {total_profit}")
        print(f"Total Number of Trades Across All Algorithms: {total_trades}")

    elif mode == 'ticker_summary':
        # Group by ticker and calculate total profit/loss and profit/loss percentage
        ticker_summary = filtered_alerts.groupby('symbol').agg(
            totalProfitOrLoss=('profitOrLoss', 'sum'),
            numberOfTrades=('symbol', 'size')
        ).reset_index()
        
        # Calculate profit/loss percentage based on total investment (assuming 500 as total investment per ticker)
        ticker_summary['profitOrLossPercentage'] = ticker_summary['totalProfitOrLoss'].apply(lambda x: f"{(x * 100 / 500):.2f}%")
        
        # Display the summary table
        display(ticker_summary)
        # Print total profit and total trades
        total_profit = ticker_summary['totalProfitOrLoss'].sum()
        total_trades = ticker_summary['numberOfTrades'].sum()
        print(f"Total Profit Across All Tickers: {total_profit}")
        print(f"Total Number of Trades Across All Tickers: {total_trades}")


In [148]:
# New mode: 'ticker_summary'
process_trades_2('project_zero.running_trades.json', ticker='MARA', exchange='SMART', algo='Zero-AI-3', interval='1', mode='ticker_summary')


,symbol,totalProfitOrLoss,numberOfTrades,profitOrLossPercentage
0,MARA,149.538,709,29.91%


Total Profit Across All Tickers: 149.53799999999535
Total Number of Trades Across All Tickers: 709
